In [1]:
'''
AIM: to detect why logqz is under estimated, thus causing negative KL-div
'''

%matplotlib inline
import os
os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cpu,floatX=float32"
import theano
import theano.tensor as T
import network
import network.normFlow as nf
import config
import utils
import utils.mathT as mathT
import utils.theanoGeneral as utilsT
import numpy as np
import utils.optimizers as optimizers
import utils.plotZ as plotZ
import matplotlib.pyplot as plt
floatX = utils.floatX

/Library/Python/2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [7]:
DIM = 2
SAMPLINGNUM = 10000

# construction variational model
iaf = nf.NormFlowModel( dim=2, numlayers=1,noisestd=1. )  #(4./DIM)**.5
e = T.fmatrix()
z, logqz = iaf.reparam(e)
fe = theano.function([e], outputs=[z,logqz])

In [3]:
w = np.asarray([[1,0],[0,1]], dtype=floatX)
b = np.asarray( [0.5,-0.5],     dtype=floatX)
u = np.asarray( [5,10 ],       dtype=floatX)
iaf.layers[0].setParamValues( w,b,u )

In [4]:
esamples = np.asarray( [[0,0],[0,1],[0.5,0.5]], dtype=floatX )
zsamples = np.tanh( b + np.dot(esamples,w) ) * u + esamples


from scipy.stats import multivariate_normal
logpes = np.log( multivariate_normal.pdf(esamples, mean=[0,0], cov=np.eye(2)) )
logpzs = np.zeros_like(logpes)
for i, logpe in enumerate(logpes):
    ei = esamples[i,:]
    det1 = 1 + u[0] * ( np.cosh(b[0]+ei[0]*w[0,0]) )**(-2)              * w[0,0]
    det2 = 1 + u[1] * ( np.cosh(b[1]+ei[0]*w[0,1]+ei[1]*w[1,1]) )**(-2) * w[1,1]
    logpzs[i] = logpes[i] - np.log( np.abs(det1*det2) )
print 'log p(e)\n', logpes
print 'log p(z)\n', logpzs
print 'zsamples\n', zsamples

log p(e)
[-1.83787707 -2.33787707 -2.08787707]
log p(z)
[-5.61572209 -6.11572209 -5.61713302]
zsamples
[[ 2.31058574 -4.62117147]
 [ 2.31058574  5.62117147]
 [ 4.307971    0.5       ]]


In [5]:
print 'zsamples by iaf model\n',fe(esamples)[0]
print 'log p(z)\n',fe(esamples)[1]

zsamples by iaf model
[[-4.62117147  2.31058574]
 [ 5.62117147  2.31058574]
 [ 0.5         4.307971  ]]
log p(z)
[-5.6157217  -6.1157217  -5.61713314]


In [4]:
# function test for mathT.gmm

means   = np.asarray( [[1.0,2.0],[1.0,2.0]] )
varmats = np.asarray( [np.eye(2)/2, np.eye(2)/2] ) 
weights = np.asarray( [0.6,1.4] )

e = T.fmatrix()
pdfNorm = mathT.multiNormInit(means[0],varmats[0])
pdfGmm  = mathT.multiGmm(means, varmats, weights)
p1 = pdfNorm(e)
p2 = pdfGmm(e)
f1 = theano.function([e],outputs=p1)
f2 = theano.function([e],outputs=p2)

In [7]:
esamples = np.asarray([[0,0],[1,1]], dtype=floatX)
print(np.exp(f1(esamples)))
print(np.exp(f2(esamples))/2)

[ 0.00214475  0.11709964]
[ 0.00214475  0.11709964]
